In [130]:
# %load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat 
from dataset import Dataset
from filter  import Filter

In [34]:
file_path_1 = "../BCICIV_1_mat/BCICIV_calib_ds1a"

In [35]:
data = loadmat(file_path_1)
[x[0] for x in data['nfo']['classes'][0][0][0]]

['left', 'foot']

In [138]:
data = Dataset(file_path_1, 8)

800

In [142]:
transformed = Filter.laplacian_filter(data)
plt.subplot(1, 2, 1)
plt.plot(data.trials[10][20])
plt.subplot(1, 2, 2)
plt.plot(transformed[10][20])
plt.show()